## Random Forest Base

### Data Preparation

In [67]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import hvplot.xarray
import sys

%matplotlib inline

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_squared_error

sys.path.insert(0, '../src')

from utils import df_to_xarray,read_xarray

In [2]:
# Reading Data
dir_name="../data/member_001"
chl,mld,sss,sst,u10,fg_co2,xco2,icefrac,patm,pco2=read_xarray(dir_name)

In [3]:
# Creating one singular df
data_read=xr.merge([mld.MLD,mld.MLD_socat,sst.SST,sst.SST_socat,sss.SSS,sss.SSS_socat,xco2])

In [98]:
tmp_data=data_read.to_dataframe().reset_index()


In [ ]:
tmp_data=tmp_data.drop(columns=['bnds','TLONG', 'TLAT', 'time_bnds'])

In [99]:
chl_data=chl.Chl.to_dataframe().reset_index()
chl_data_socat=chl.Chl_socat.to_dataframe().reset_index()
pco2_data=pco2.pCO2.to_dataframe().reset_index()
pco2_data_socat=pco2.pCO2_socat.to_dataframe().reset_index()

In [100]:
tmp_data["Chl_socat"]=chl_data_socat["Chl_socat"]
tmp_data["Chl"]=chl_data["Chl"]
tmp_data["pCO2_socat"]=pco2_data_socat["pCO2_socat"]
tmp_data["pCO2"]=pco2_data["pCO2"]

In [101]:
features_socat = ['time','xlon', 'ylat','MLD_socat', 'SST_socat', 'SSS_socat','Chl_socat', 'XCO2','pCO2_socat']
features = ['time','xlon', 'ylat','MLD','SST','SSS','Chl','XCO2','pCO2']

# create separate dataframe for socat
combined_socat=tmp_data.loc[:,features_socat]
combined=tmp_data.loc[:,features]

In [102]:
# drop rows where pco2 or pco2_socat == NA or 0
combined_socat.dropna(subset = ["pCO2_socat"], inplace=True)
combined_socat= combined_socat[combined_socat['pCO2_socat']!=0]
combined.dropna(subset = ["pCO2"], inplace=True)
combined= combined[combined['pCO2']!=0]



In [103]:
#separating X and y
X_socat=combined_socat.iloc[:,3:-1]
X=combined.iloc[:,3:-1]
y=combined.loc[:,'pCO2']
y_socat=combined_socat.loc[:,'pCO2_socat']


### Imputation
We can save 6452246 rows through imputation.

Try Building a Custom Imputation based on lon and lat?
https://towardsdatascience.com/coding-a-custom-imputer-in-scikit-learn-31bd68e541de


Also, consider using Hurdle Model?

https://geoffruddock.com/building-a-hurdle-regression-estimator-in-scikit-learn/

#### Two Different Imputation Methods
- KNNImputer: fill in the average of the 2 nearest neighbors, takes a long time to train
- Simple Imputer: just fill in using the average

In [23]:
X_socat.describe()

,MLD_socat,SST_socat,SSS_socat,Chl_socat,XCO2
count,132903.000000,132903.000000,132903.000000,245577.000000,245577.000000
mean,1.023873,0.305430,0.659811,0.365122,382.684174
std,10.095239,2.532298,4.656825,0.820338,15.272440
min,0.000000,-1.825340,0.000000,-0.089630,340.848541
25%,0.000000,0.000000,0.000000,0.124559,371.568420
50%,0.000000,0.000000,0.000000,0.178825,384.370575
75%,0.000000,0.000000,0.000000,0.222653,394.913391
max,1193.783569,30.828823,36.894070,13.049178,407.208405


In [25]:
X.describe()

,MLD,SST,SSS,Chl,XCO2
count,1.083822e+07,1.083822e+07,1.083822e+07,1.729047e+07,1.729047e+07
mean,6.533868e+01,1.319081e+01,3.375592e+01,3.449500e-01,3.701860e+02
std,5.373071e+01,1.153312e+01,1.569614e+00,8.521562e-01,1.869547e+01
min,7.500032e+00,-1.936021e+00,1.376396e+01,-4.092084e-01,3.408485e+02
25%,3.466645e+01,6.017284e-01,3.324879e+01,1.087646e-01,3.547707e+02
50%,5.551451e+01,1.352542e+01,3.385484e+01,1.580932e-01,3.681608e+02
75%,8.369460e+01,2.482988e+01,3.459950e+01,2.111136e-01,3.854302e+02
max,1.868325e+03,3.407636e+01,4.305632e+01,1.467028e+01,4.072084e+02


In [174]:
#We can save this many rows through imputation.
# These rows have xCO2, pXO2 and CHL, but no MLD, SSS, SST
combined_socat.isna().sum()

time               0
xlon               0
ylat               0
MLD_socat     112674
SST_socat     112674
SSS_socat     112674
Chl_socat          0
XCO2               0
pCO2_socat         0
dtype: int64

In [26]:
# Two Different Imputation Methods

# KNNImputer
# from sklearn.impute import KNNImputer
# imp = KNNImputer(n_neighbors=2)
# X=imp.fit_transform(X)
# X_socat=imp.fit_transform(X_socat)

# SimpleImputer
from sklearn.impute import SimpleImputer
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
X=imp.fit_transform(X)
X_socat=imp.fit_transform(X_socat)

### Modeling - Random Forest


Uses train_test_split build into sklearn.model_selection


By default this method shuffles the data (30% = testing 70%=training/validation)
- Will test validation via 7-fold cross validation

Train  = 70%, Test   = 30%

In [27]:

X_train, X_test, y_train, y_test = train_test_split(X_socat, y_socat, test_size=0.3, random_state= 73)


In [28]:
regressor=RandomForestRegressor(n_estimators=20, random_state=42)
regressor.fit(X_train, y_train)

RandomForestRegressor(n_estimators=20, random_state=42)

In [29]:
from sklearn.model_selection import cross_val_score

def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

forest_scores = cross_val_score(regressor, X_train, y_train,
                                scoring="neg_mean_squared_error", cv=7)
forest_rmse_scores = np.sqrt(-forest_scores)

In [30]:
display_scores(forest_rmse_scores)

Scores: [31.30501344 30.48264106 30.56466388 30.81977593 31.88391381 30.6039626
 30.76351001]
Mean: 30.917640103017195
Standard deviation: 0.46740719704756006


In [31]:
#Fine Tuning Using RandomizedSearch

from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_distribs = {
        'n_estimators': randint(low=20, high=50),
        'max_features': randint(low=1, high=6),
    }

forest_reg = RandomForestRegressor(random_state=42)
rnd_search = RandomizedSearchCV(forest_reg, param_distributions=param_distribs,
                                n_iter=10, cv=7, scoring='neg_mean_squared_error', random_state=42)
rnd_search.fit(X_train, y_train)

RandomizedSearchCV(cv=7, estimator=RandomForestRegressor(random_state=42),
                   param_distributions={'max_features': <scipy.stats._distn_infrastructure.rv_frozen object at 0x13efec250>,
                                        'n_estimators': <scipy.stats._distn_infrastructure.rv_frozen object at 0x13efec4f0>},
                   random_state=42, scoring='neg_mean_squared_error')

In [32]:
final_model =rnd_search.best_estimator_


### Final Result

Test Set RMSE: 30.56

Whole Grid Rmse: 42.12

In [33]:
y_pred=final_model.predict(X_test)
test_mse=mean_squared_error(y_test, y_pred,squared=True)
np.sqrt(test_mse)

30.55997229013893

In [56]:
## The whole grid
y_pred=final_model.predict(X)
final_test_rmse=np.sqrt(mean_squared_error(y, y_pred,squared=True))
error=y-y_pred

In [35]:
final_test_rmse

42.11730847320173

### Visualization of the Residual

In [104]:
combined["residual"]=np.abs(error)

In [105]:
result_data=combined[["time","xlon","ylat","residual"]]

In [106]:
cols=result_data.columns.tolist()
cols=[cols[0],cols[2],cols[1],cols[3]]
result_data=result_data[cols]
result_data.columns=['time','lat','lon','residual']

In [107]:
result_data['time'].iloc[0]

cftime.DatetimeNoLeap(1982, 2, 1, 0, 0, 0, 0, has_year_zero=True)

In [69]:
ds=df_to_xarray(result_data[:100])

# p="../data/rf_residuals.nc"
# ds.to_netcdf(path=p)

In [145]:
a=result_data[:100]



In [171]:

def tmp_df(df_in=None):
    dates = xr.cftime_range(start=f'1982-02-01', end=f'2018-12-01',freq='MS') 
    ds_skeleton = xr.Dataset({'lon':np.arange(0.5, 360, 1), 
                              'lat':np.arange(-89.5, 90, 1),
                              'time':dates})    
    # make dataframe
    skeleton = df_in.reset_index()[['time','lat','lon']]
    # Merge predictions with df_all dataframe
    df_out = skeleton.merge(df_in, how = 'left', on = ['time','lat','lon'])
    # convert to xarray dataset
    # old way to `dimt, = ds_skeleton.time.shape` ect. to get dimensions
    # then reshape  `df_out.values.reshape(dim_lat, dim_lon, dim_time)`
    # finally create a custom dataset
    df_out.set_index(['time', 'lat','lon'], inplace=True)
    ds = df_out.to_xarray()
    return ds

ds=tmp_df(result_data)
              


<xarray.DataArray 'residual' (time: 421, lat: 180, lon: 336)>
array([[[         nan, 149.33411541,          nan, ...,   0.28033776,
           3.45040735,   2.02060836],
        [         nan, 144.62794434,          nan, ...,   0.2869374 ,
           3.45283642,   2.02162333],
        [         nan, 153.0305287 ,          nan, ...,   0.2936794 ,
           3.45521164,   2.0226871 ],
        ...,
        [         nan,          nan, 144.41993015, ...,  29.52223072,
          31.84785863,  30.42458839],
        [         nan,          nan, 146.58182344, ...,  29.50957832,
          31.84842479,  30.42197143],
        [147.62288479,          nan, 150.16147704, ...,  29.49713174,
          31.84756848,  30.41935588]],

       [[         nan, 149.24853442,          nan, ...,   7.59816208,
           8.21080611,  10.36075988],
        [         nan, 155.07162028,          nan, ...,   7.58157273,
           8.20399911,  10.37029497],
        [         nan, 136.23161331,          nan, ...,   7.56494288,
           8.19729447,  10.37970754],
...
        [         nan,          nan,  35.31923461, ...,  56.78145232,
          62.18874345,  59.84878685],
        [         nan,          nan,  36.17601058, ...,  56.69317361,
          62.20735353,  59.82151069],
        [ 96.55323057,          nan,  38.67487147, ...,  56.60582022,
          62.22708529,  59.79452993]],

       [[         nan, 197.61239218,          nan, ...,  43.38401463,
          40.09840641,  41.99308284],
        [         nan, 181.91874696,          nan, ...,  43.44474888,
          40.07217359,  42.01865324],
        [         nan, 188.27622477,          nan, ...,  43.50506058,
          40.04615771,  42.04400862],
        ...,
        [         nan,          nan, 151.4096125 , ...,  36.99084894,
          40.74461935,  39.00526698],
        [         nan,          nan, 189.44916769, ...,  36.9182199 ,
          40.76793302,  38.97839543],
        [200.38060064,          nan, 222.12920871, ...,  36.84617717,
          40.79404426,  38.95167664]]])
Coordinates:
  * time     (time) object 1982-02-01 00:00:00 ... 2017-02-01 00:00:00
  * lat      (lat) float64 -89.5 -88.5 -87.5 -86.5 -85.5 ... 86.5 87.5 88.5 89.5
  * lon      (lon) float64 24.5 25.5 26.5 27.5 28.5 ... 356.5 357.5 358.5 359.5

In [173]:
a=ds.residual.hvplot(groupby="time",width=512,height=512, widget_type='scrubber', widget_location='bottom')
a

Column
    [0] HoloViews(DynamicMap, widget_location='bottom', widget_type='scrubber')
    [1] Row
        [0] HSpacer()
        [1] WidgetBox
            [0] Player(end=420, width=550)
        [2] HSpacer()